## Project Overview

**Real-time fire detection system using YOLOv8n, optimized with threshold tuning for webcam inference and Desktop video recording.**

## Dataset Composition
**Source**: [Roboflow – continuous_fire](https://universe.roboflow.com/-jwzpw/continuous_fire)

**Train set**: train/images + train/labels — fire images with YOLO TXT annotations (single class: fire)

**Validation set**: valid/images + valid/labels — held-out validation images for hyperparameter and threshold tuning

**Test set**: test/images + test/labels — final evaluation split used to report precision, recall, mAP50, and mAP50-95

**Class definitions**
- nc: 1
- names: [fire]

### 1. Import Libraries

In [1]:
import os, json
import torch
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from IPython.display import Image, display

### 2. YOLOv8 Fire Detection Training & Evaluation
- Trains YOLOv8n for single-class fire detection with auto-generated data.yaml.
- Includes GPU→CPU fallback and logs key metrics per epoch (loss, precision, recall, mAP).
- Output:
    - Train: P: 0.785, R: 0.727, mAP50: 0.767, mAP50–95: 0.516
    - Validation: P: 0.783, R: 0.726, mAP50: 0.769, mAP50–95: 0.516
    - Test: P: 0.791, R: 0.655, mAP50: 0.720, mAP50–95: 0.487

In [2]:
# Reduce CUDA memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# Dataset paths (adjust if needed)
ROOT  = Path(".\continuous_fire.v5i.yolov8")
TRAIN = (ROOT / "train" / "images").as_posix()
VAL   = (ROOT / "valid" / "images").as_posix()
TEST  = (ROOT / "test"  / "images").as_posix()

# Auto-generate data.yaml (single class: fire)
runs_dir  = Path("runs"); runs_dir.mkdir(exist_ok=True, parents=True)
yaml_path = runs_dir / "dataset_fire.yaml"
yaml_path.write_text(
    f"train: {TRAIN}\nval: {VAL}\ntest: {TEST}\n"
    f"nc: 1\nnames: [fire]\n",
    encoding="utf-8"
)

# Hyperparameters
MODEL_WEIGHTS = "yolov8n.pt"
IMGSZ, BATCH, FREEZE, EPOCHS, PATIENCE = 320, 1, 1, 11, 3
NAME, DEVICE_PRIMARY = "fire_lowmem_single", 0  # 0 = GPU, "cpu" = CPU

# Simple callback: print key metrics per epoch
def on_fit_epoch_end(trainer):
    m = trainer.metrics or {}
    print(f"[Epoch {trainer.epoch+1}/{trainer.epochs}] "
          f"box:{m.get('train/box_loss', float('nan')):.3f} "
          f"cls:{m.get('train/cls_loss', float('nan')):.3f} "
          f"dfl:{m.get('train/dfl_loss', float('nan')):.3f} | "
          f"P:{m.get('metrics/precision(B)', float('nan')):.3f} "
          f"R:{m.get('metrics/recall(B)', float('nan')):.3f} "
          f"mAP50:{m.get('metrics/mAP50(B)', float('nan')):.3f} "
          f"mAP50-95:{m.get('metrics/mAP50-95(B)', float('nan')):.3f}")

# Train (GPU→CPU fallback)
def train(device):
    model = YOLO(MODEL_WEIGHTS)
    model.add_callback("on_fit_epoch_end", on_fit_epoch_end)
    return model.train(
        data=str(yaml_path), epochs=EPOCHS, imgsz=IMGSZ, batch=BATCH,
        workers=0, device=device, cache=False, freeze=FREEZE, patience=PATIENCE,
        project="runs", name=NAME, plots=True
    )

try:
    results = train(device=DEVICE_PRIMARY)
except RuntimeError as e:
    print("[WARN] CUDA error, fallback to CPU:", e)
    torch.cuda.empty_cache()
    results = train(device="cpu")

# Reuse these in next blocks
save_dir      = Path(results.save_dir)
best_pt_path  = save_dir / "weights" / "best.pt"

# Quick evaluation on val & test
best_model = YOLO(best_pt_path.as_posix())
best_model.val(data=str(yaml_path))
try:
    best_model.val(data=str(yaml_path), split="test")
except TypeError:
    best_model.predict(source=TEST, imgsz=IMGSZ, save=True,
                       project="runs", name=f"{NAME}_test_pred")
print(f"[INFO] Trained weights: {best_pt_path}")


100%|█████████████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:00<00:00, 10.3MB/s]


New https://pypi.org/project/ultralytics/8.3.203 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.0  Python-3.11.9 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce MX450, 2048MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=runs\dataset_fire.yaml, epochs=11, time=None, patience=3, batch=1, imgsz=320, save=True, save_period=-1, cache=False, device=0, workers=0, project=runs, name=fire_lowmem_single, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=1, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save

train: Scanning C:\Users\82109\Desktop\continuous_fire.v5i.yolov8\train\labels.cache... 2980 images, 6 backgrounds, 0 c
val: Scanning C:\Users\82109\Desktop\continuous_fire.v5i.yolov8\valid\labels.cache... 754 images, 2 backgrounds, 0 corr


Plotting labels to runs\fire_lowmem_single\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 63 weight(decay=0.0), 70 weight(decay=0.0005), 69 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 320 train, 320 val
Using 0 dataloader workers
Logging results to runs\fire_lowmem_single
Starting training for 11 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/11      0.17G      1.605      2.233        1.4          1        320: 100%|██████████| 2980/2980 [04:09<00:00,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 377/377 [00:


...

                   all        754        950      0.785      0.727      0.767      0.516
Speed: 0.3ms preprocess, 7.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\fire_lowmem_single
[Epoch 11/11] box:nan cls:nan dfl:nan | P:0.785 R:0.727 mAP50:0.767 mAP50-95:0.516
Ultralytics 8.3.0  Python-3.11.9 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce MX450, 2048MiB)
Model summary (fused): 186 layers, 2,684,563 parameters, 0 gradients, 6.8 GFLOPs


val: Scanning C:\Users\82109\Desktop\continuous_fire.v5i.yolov8\valid\labels.cache... 754 images, 2 backgrounds, 0 corr
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 48/48 [00:05


                   all        754        950      0.783      0.726      0.769      0.516
Speed: 0.5ms preprocess, 3.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\val
Ultralytics 8.3.0  Python-3.11.9 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce MX450, 2048MiB)


val: Scanning C:\Users\82109\Desktop\continuous_fire.v5i.yolov8\test\labels.cache... 751 images, 2 backgrounds, 0 corru
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:06


                   all        751        963      0.791      0.655       0.72      0.487
Speed: 0.3ms preprocess, 3.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\val2
[INFO] Trained weights: runs\fire_lowmem_single\weights\best.pt


### 3. Confidence/IoU Threshold Sweep

- Runs a confidence sweep (0.05–0.80) to evaluate precision, recall, and F1.
- Selects optimal confidence based on F1 score or target recall, then applies it for deployment.
- Output:
    - Best threshold chosen: conf = 0.05, iou = 0.55
    - Validation: P=0.854, R=0.735, F1=0.790, mAP50=0.823, mAP50–95=0.570
    - Test: P=0.863, R=0.659, F1=0.747, mAP50=0.778, mAP50–95=0.543

In [3]:
model = YOLO(best_pt_path.as_posix())
CONF_LIST     = np.linspace(0.05, 0.80, 3)
IOU_NMS       = 0.55
TARGET_RECALL = 0.80   # set None to pick by F1 only

def eval_split(split):
    rows = []
    for c in CONF_LIST:
        r = model.val(data=str(yaml_path), split=split, conf=float(c), iou=IOU_NMS,
                      plots=False, verbose=False)
        d = r.results_dict
        P = float(d.get("metrics/precision(B)", float("nan")))
        R = float(d.get("metrics/recall(B)",    float("nan")))
        m50  = float(d.get("metrics/mAP50(B)",     float("nan")))
        m959 = float(d.get("metrics/mAP50-95(B)",  float("nan")))
        F1 = (2*P*R/(P+R)) if (P+R)>0 else float("nan")
        rows.append({"conf": float(c), "precision": P, "recall": R,
                     "F1": F1, "mAP50": m50, "mAP50-95": m959})
    return rows

def pick_best(rows, target_recall=None):
    if target_recall is None:
        return max(rows, key=lambda x: x["F1"])
    cand = [r for r in rows if r["recall"] >= target_recall]
    return max(cand, key=lambda x: x["precision"]) if cand else max(rows, key=lambda x: x["recall"])

val_rows  = eval_split("val");   best_val  = pick_best(val_rows,  TARGET_RECALL)
test_rows = eval_split("test");  best_test = pick_best(test_rows, TARGET_RECALL)

BEST_CONF = best_val["conf"]  # choose on val; swap to best_test if preferred
print("[VAL]  best =", best_val)
print("[TEST] best =", best_test)
print(f"[THRESH] Use conf={BEST_CONF:.3f}, iou={IOU_NMS:.2f}")

# Persist chosen thresholds for later use
th_dir = Path("runs/thresholds"); th_dir.mkdir(parents=True, exist_ok=True)
with open(th_dir / "fire_best_threshold.json", "w") as f:
    json.dump({"conf": BEST_CONF, "iou": IOU_NMS}, f, indent=2)


Ultralytics 8.3.0  Python-3.11.9 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce MX450, 2048MiB)
Model summary (fused): 186 layers, 2,684,563 parameters, 0 gradients, 6.8 GFLOPs


val: Scanning C:\Users\82109\Desktop\continuous_fire.v5i.yolov8\valid\labels.cache... 754 images, 2 backgrounds, 0 corr
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 48/48 [00:04

                   all        754        950      0.854      0.735      0.823       0.57
Speed: 0.4ms preprocess, 3.0ms inference, 0.0ms loss, 0.8ms postprocess per image
Ultralytics 8.3.0  Python-3.11.9 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce MX450, 2048MiB)



val: Scanning C:\Users\82109\Desktop\continuous_fire.v5i.yolov8\valid\labels.cache... 754 images, 2 backgrounds, 0 corr
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 48/48 [00:04

                   all        754        950      0.902      0.675      0.787      0.577
Speed: 0.3ms preprocess, 3.2ms inference, 0.0ms loss, 0.7ms postprocess per image
Ultralytics 8.3.0  Python-3.11.9 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce MX450, 2048MiB)


...


val: Scanning C:\Users\82109\Desktop\continuous_fire.v5i.yolov8\test\labels.cache... 751 images, 2 backgrounds, 0 corru
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:04

                   all        751        963      0.898       0.62      0.752      0.552
Speed: 0.5ms preprocess, 3.0ms inference, 0.0ms loss, 0.7ms postprocess per image
Ultralytics 8.3.0  Python-3.11.9 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce MX450, 2048MiB)



val: Scanning C:\Users\82109\Desktop\continuous_fire.v5i.yolov8\test\labels.cache... 751 images, 2 backgrounds, 0 corru
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:04

                   all        751        963       0.93      0.305      0.609      0.479


Speed: 0.5ms preprocess, 3.0ms inference, 0.0ms loss, 0.6ms postprocess per image
[VAL]  best = {'conf': 0.05, 'precision': 0.8542189214430793, 'recall': 0.7347368421052631, 'F1': 0.7899856335913117, 'mAP50': 0.8230600085636043, 'mAP50-95': 0.57002070295112}
[TEST] best = {'conf': 0.05, 'precision': 0.8625845246422935, 'recall': 0.6593977154724818, 'F1': 0.7474282550210991, 'mAP50': 0.7782427869398519, 'mAP50-95': 0.5434087933870979}
[THRESH] Use conf=0.050, iou=0.55


### 4. Webcam Inference with Best Threshold
- Applies the optimal threshold (conf/IoU) for real-time webcam detection.

In [6]:
import time
import cv2

# Fixed thresholds from sweep 
IMGSZ     = 320
DEVICE    = 0          
BEST_CONF = 0.05       # [VAL] best
IOU_NMS   = 0.55

print(f"[THRESH] Using conf={BEST_CONF:.3f}, iou={IOU_NMS:.2f}")

# Load model (.pt path must exist) 
model = YOLO(Path(best_pt_path).as_posix())

# Open webcam 
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)   
cap.set(cv2.CAP_PROP_FRAME_WIDTH,  640)    
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_BUFFERSIZE,   1)

win_name = f"YOLO Fire Detection | conf={BEST_CONF:.2f}, iou={IOU_NMS:.2f} | press 'q' to quit"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)

prev_t = time.time()
fps = 0.0

with torch.no_grad():
    while True:
        ok, frame = cap.read()
        if not ok:
            break

        results = model(
            frame,
            imgsz=IMGSZ,
            conf=BEST_CONF,
            iou=IOU_NMS,
            device=DEVICE
        )
        annotated = results[0].plot()

        now = time.time()
        dt  = now - prev_t
        if dt > 0:
            fps = 1.0 / dt
        prev_t = now
        cv2.putText(annotated, f"FPS: {fps:.1f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow(win_name, annotated)

        # Quit on 'q'
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()


[THRESH] Using conf=0.050, iou=0.55

0: 256x320 (no detections), 75.7ms
Speed: 2.0ms preprocess, 75.7ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 10.1ms
Speed: 1.0ms preprocess, 10.1ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 8.0ms
Speed: 0.9ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 8.9ms
Speed: 1.1ms preprocess, 8.9ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 11.5ms
Speed: 0.0ms preprocess, 11.5ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 11.9ms
Speed: 1.4ms preprocess, 11.9ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 8.8ms
Speed: 0.0ms preprocess, 8.8ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 14.0ms
Spe


0: 256x320 (no detections), 9.9ms
Speed: 1.1ms preprocess, 9.9ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 10.0ms
Speed: 1.5ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 11.1ms
Speed: 0.9ms preprocess, 11.1ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.4ms
Speed: 1.5ms preprocess, 12.4ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 14.5ms
Speed: 1.0ms preprocess, 14.5ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 14.0ms
Speed: 1.0ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 10.7ms
Speed: 1.0ms preprocess, 10.7ms inf


0: 256x320 (no detections), 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 11.5ms
Speed: 1.5ms preprocess, 11.5ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 11.9ms
Speed: 1.0ms preprocess, 11.9ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 10.3ms
Speed: 1.0ms preprocess, 10.3ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 14.9ms
Speed: 1.0ms preprocess, 14.9ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 11.0ms
Speed: 0.8ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 2.0ms preprocess, 12.0ms i


0: 256x320 (no detections), 11.6ms
Speed: 1.0ms preprocess, 11.6ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 11.7ms
Speed: 1.3ms preprocess, 11.7ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 11.5ms
Speed: 1.0ms preprocess, 11.5ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.6ms
Speed: 1.0ms preprocess, 12.6ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.3ms
Speed: 1.0ms preprocess, 12.3ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 11.5ms
Speed: 1.0ms preprocess, 11.5ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 13.0ms
Speed: 1.0ms preprocess, 13.0ms i


0: 256x320 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 11.2ms
Speed: 1.1ms preprocess, 11.2ms inference, 1.6ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 11.5ms
Speed: 1.0ms preprocess, 11.5ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 13.3ms
Speed: 2.0ms preprocess, 13.3ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 13.0ms
Speed: 1.0ms preprocess, 13.0ms i

...


0: 256x320 1 fire, 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 1 fire, 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 1 fire, 11.9ms
Speed: 1.1ms preprocess, 11.9ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 1 fire, 12.4ms
Speed: 1.0ms preprocess, 12.4ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 1 fire, 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 1 fire, 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 1 fire, 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 2.5ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 1 fire, 12.2ms
Speed: 2.1ms preprocess, 12.2ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x

Speed: 1.0ms preprocess, 14.1ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.4ms
Speed: 1.0ms preprocess, 12.4ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 13.1ms
Speed: 1.0ms preprocess, 13.1ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.5ms
Speed: 1.0ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0ms postprocess per imag


0: 256x320 (no detections), 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.2ms
Speed: 1.0ms preprocess, 12.2ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.1ms
Speed: 1.0ms preprocess, 12.1ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 14.3ms
Speed: 2.0ms preprocess, 14.3ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 0.9ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.5ms
Speed: 1.0ms preprocess, 12.5ms i


0: 256x320 (no detections), 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 13.0ms
Speed: 0.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 0.9ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.5ms
Speed: 1.0ms preprocess, 12.5ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 2.0ms preprocess, 12.0ms i


0: 256x320 (no detections), 12.1ms
Speed: 0.9ms preprocess, 12.1ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 13.3ms
Speed: 1.0ms preprocess, 13.3ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.2ms
Speed: 1.0ms preprocess, 12.2ms inference, 1.1ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.5ms
Speed: 1.0ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 1.1ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 (no detections), 12.1ms
Speed: 1.0ms preprocess, 12.1ms i